### importing libraries

In [1]:
from tensorflow.keras import layers, models, datasets, Model
from PIL import Image

2025-07-31 01:42:04.418339: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-31 01:42:04.621720: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-31 01:42:04.703345: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753936924.854290   30554 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753936924.890594   30554 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753936925.164006   30554 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

### loading data

In [ ]:

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# Setting input shape, normalizing color channel, setting datatype to float32 for numerical stability
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

classes = [str(i) for i in range(10)]

In [3]:

folders = [entry.name for entry in os.scandir(BASE_PATH)]

for folder_name in folders:
  print(folder_name)

  image_files = [f"{folder_name}/{f}" for f in os.listdir(SOURCE_DIR+folder_name) if f.endswith(('.png'))]

  # Use ProcessPoolExecutor to run augmentations on all available CPU cores
  with ProcessPoolExecutor() as executor:
      results = executor.map(augment_and_save, image_files)

  # total_saved = sum(results)
  # print(f"\nGenerated a total of {total_saved} augmented images for class")

0
1
2
3
4
5
6
7
8
9
A
B
C
D
E
F
G
H
I
J
K
L
M
N
P
Q
R
S
T
U
V
W
X
Y
Z
augmented_data


### Data preprocessing 

#### Image size reduction

In [ ]:
import cv2
import numpy as np

# Assume 'original_image' is a 150x200 numpy array
new_size = (40, 30) # Note: OpenCV uses (width, height) format

# Resize the image
# cv2.INTER_AREA is best for shrinking images
resized_image = cv2.resize(original_image, new_size, interpolation=cv2.INTER_AREA)

# resized_image is now a 30x40 numpy array

#### Thresholding

In [ ]:
import cv2

# Otsu's Thresholding (Recommended Start)
# cv2.THRESH_BINARY_INV makes the character white (255) and background black (0)
otsu_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Adaptive Thresholding (Most Robust)
# Parameters like block size (e.g., 11) and C (a constant) may need tuning.
adaptive_img = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                     cv2.THRESH_BINARY_INV, 11, 2)

#### Data augmentation

In [2]:
import os
import cv2
import albumentations as A
from concurrent.futures import ProcessPoolExecutor

# --- Configuration ---
BASE_PATH = "./data/train/"
SOURCE_DIR = BASE_PATH
OUTPUT_DIR = './data/train/augmented_data'
NUM_AUGMENTATIONS_PER_IMAGE = 10 # How many new versions to create for each original

# These are good augmentations for character OCR. They are applied with some probability.
transform = A.Compose([
    A.Affine(
        translate_percent=0.05,
        scale=(0.95, 1.05),
        rotate=(-10, 10),
        p=0.8
    ),
    A.MotionBlur(blur_limit=5, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.7),
])

def augment_and_save(image_filename):
    # Create output directory if it doesn't exist
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    filepath = os.path.join(SOURCE_DIR, image_filename)
    try:
        # Read the image using OpenCV
        image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Warning: Could not read {image_filename}, skipping.")
            return 0

        # Generate and save augmented images
        saved_count = 0
        base_filename = os.path.splitext(image_filename)[0]
        
        for i in range(NUM_AUGMENTATIONS_PER_IMAGE):
            augmented = transform(image=image)
            augmented_image = augmented['image']

            new_filename = f"{base_filename}_aug_{i}.png"
            output_path = os.path.join(OUTPUT_DIR, new_filename)
            
            # Create subdirectory if it doesn't exist
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
            cv2.imwrite(output_path, augmented_image)
            saved_count += 1
        return saved_count
    except Exception as e:
        print(f"Error processing {image_filename}: {e}")
        return 0


### architecture

Input -> Conv2D -> ReLU -> MaxPool2D -> Conv2D -> ReLU -> MaxPool2D -> Flatten -> Dense -> ReLU -> Dense (Output) -> Softmax
Use batch normalization
IF SIGNS OF OVERFITTING: DROPOUT. IF OVER REGULARIZED: L2 REG (WD) 

In [3]:
#@title Model: convolutional layers

model = models.Sequential()


model.add(layers.Conv2D(24, (3, 3), activation = 'relu', input_shape = (28, 28, 1))) # -> (25,25,1)
model.add(layers.MaxPooling2D((2, 2))) # -> (23,23,1)

model.add(layers.Conv2D(48, (3, 3), strides=(2,2), activation = 'relu')) # -> (10,10,1)
model.add(layers.MaxPooling2D((2, 2))) # -> (8,8,1)


c:\Users\Enzo Racciatti\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [4]:
#@title Model: Fully connected layers

model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(32, activation = 'softmax'))

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 24)     │           240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 24)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 6, 6, 48)       │        10,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 3, 3, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 432)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        27,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,448 (158.00 KB)

 Trainable params: 40,448 (158.00 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
#@title Compilação
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

### training


#### early stopping config

In [ ]:

from tensorflow.keras.callbacks import EarlyStopping


# Configure the callback
# 'monitor' is the metric to watch.
# 'patience' is the number of epochs to wait for improvement.
# 'restore_best_weights=True' automatically loads the model weights from the best epoch.

early_stopper = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#### Learning rate scheduler config

In [ ]:

from tensorflow.keras.callbacks import ReduceLROnPlateau
# Configure the callback
lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
                                 factor=0.2, # Factor by which to reduce the LR: new_lr = lr * factor
                                 patience=2, # Number of epochs with no improvement to wait
                                 min_lr=0.00001) # Lower bound on the learning rate



#### Traning itself

In [ ]:

history = model.fit(train_images,
                    train_labels,
                    epochs = 1,
                    validation_data = (test_images, test_labels))



# Pass the callback to model.fit()
# Make sure to provide validation data!
model.fit(train_ds,
          epochs=100, # Set a high number, early stopping will find the right time.
          validation_data=val_ds,
          callbacks=[early_stopper]) # Pass it in a list

### Evaluation

In [8]:
test_error, test_acc = model.evaluate(test_images, test_labels, verbose=2)

313/313 - 2s - 8ms/step - accuracy: 0.9860 - loss: 0.1554


### Post-training efficiency increase
Quantization, ...